In [1]:
!pip install pandas


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [4]:
import pandas as pd

# Load the CSV files
updated_livelab = pd.read_csv('Updated_LiveLab.csv')
marked_help_instances = pd.read_csv('Marked Help Instances - 7x - anon - Marked Help Instances - 7x - anon.csv')

# Convert time columns to datetime
updated_livelab['Time'] = pd.to_datetime(updated_livelab['Time'])
marked_help_instances['helped_server_time2'] = pd.to_datetime(marked_help_instances['helped_server_time2'])

# Filter rows where helpedTransaction is True
helped_transactions = updated_livelab[updated_livelab['helpedTransaction'] == True]

# Initialize a list to store time differences
time_differences = []

# Iterate over each student
for student_id in helped_transactions['Anon Student Id'].unique():
    # Get all helpedTransaction times for the current student
    student_helped_times = helped_transactions[helped_transactions['Anon Student Id'] == student_id]['Time']
    
    # Get all marked_help_instance times for the current student
    student_marked_times = marked_help_instances[marked_help_instances['anon_student_id'] == student_id]['helped_server_time2']
    
    # For each helpedTransaction time, find the closest marked_help_instance time
    for helped_time in student_helped_times:
        # Calculate the absolute time differences between helped_time and all marked_help_instance times
        time_diffs = (student_marked_times - helped_time).abs()
        
        # Find the minimum time difference (closest marked_help_instance time)
        min_diff = time_diffs.min()
        
        # Append the minimum difference to the list
        time_differences.append(min_diff)

print(time_differences)
# Convert the list of time differences to a pandas Series for easier calculations
time_differences_series = pd.Series(time_differences)

# Calculate the average time difference (fixed time window)
fixed_time_window = time_differences_series.mean()

# Print the result
print(f"The average time difference is: {fixed_time_window}")

[Timedelta('0 days 00:00:03'), Timedelta('0 days 00:00:03'), Timedelta('0 days 00:00:10'), Timedelta('0 days 00:00:16'), Timedelta('0 days 00:00:01'), Timedelta('0 days 00:03:23'), Timedelta('0 days 00:01:58'), Timedelta('0 days 00:00:01'), Timedelta('0 days 00:00:07'), Timedelta('0 days 00:00:10'), Timedelta('0 days 00:00:10'), Timedelta('4 days 23:45:05'), Timedelta('0 days 00:00:07'), Timedelta('5 days 01:23:57'), Timedelta('0 days 00:00:02'), Timedelta('0 days 00:00:14'), Timedelta('0 days 00:00:14'), Timedelta('0 days 00:00:01'), Timedelta('0 days 00:00:10'), Timedelta('0 days 00:00:07'), Timedelta('0 days 00:00:09'), Timedelta('0 days 00:23:00'), Timedelta('7 days 23:49:07'), Timedelta('0 days 00:05:54'), Timedelta('0 days 00:00:10'), Timedelta('0 days 00:00:02'), Timedelta('0 days 00:00:31'), Timedelta('0 days 00:01:32'), Timedelta('0 days 00:02:14'), Timedelta('0 days 00:00:45'), Timedelta('0 days 00:00:08'), Timedelta('0 days 00:01:33'), Timedelta('0 days 00:00:40'), Timedelta

In [5]:
print(fixed_time_window)

2 days 11:39:29.971563981


In [10]:
# Iterate over each student to define the pre-help window and mark transactions
for student_id in updated_livelab['Anon Student Id'].unique():
    # Get all transactions for the current student
    student_transactions = updated_livelab[updated_livelab['Anon Student Id'] == student_id]
    
    # Get all marked_help_instance times for the current student
    student_marked_times = marked_help_instances[marked_help_instances['anon_student_id'] == student_id]['helped_server_time2']
    
    # Get all helpedTransaction times for the current student
    student_helped_times = student_transactions[student_transactions['helpedTransaction'] == True]['Time']
    
    # For each helpedTransaction time, define the pre-help window
    for helped_time in student_helped_times:
        # Define the pre-help window
        pre_help_start = helped_time - fixed_time_window
        pre_help_end = helped_time
        
        # Find the closest marked_help_instance time to the helped_time
        closest_marked_time = (student_marked_times - helped_time).abs().idxmin()
        # Find the transaction in updated_livelab closest to the marked_help_instance time
        closest_transaction_index = (student_transactions['Time'] - student_marked_times.loc[closest_marked_time]).abs().idxmin()
        closest_problem_name = student_transactions.loc[closest_transaction_index, 'Problem Name']
        
        # Mark transactions within the pre-help window and with the same problem name
        updated_livelab.loc[
            (updated_livelab['Anon Student Id'] == student_id) &
            (updated_livelab['Time'] >= pre_help_start) &
            (updated_livelab['Time'] <= pre_help_end) &
            (updated_livelab['Problem Name'] == closest_problem_name),
            'helpedTransaction'
        ] = True

# Save the updated dataframe to a new CSV file
updated_livelab.to_csv('PreHelp_LiveLab.csv', index=False)

# Display the updated DataFrame
print(updated_livelab["helpedTransaction"])

0          False
1          False
2          False
3          False
4          False
           ...  
1650341    False
1650342    False
1650343    False
1650344    False
1650345    False
Name: helpedTransaction, Length: 1650346, dtype: bool
